## Usage
- Cell -> Run All.
- A Napari window will open, where you can scroll through your data.

## Code
You don't need to make changes in the code cells below. 

In [1]:
import dask.array as da
import glob
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import napari
import numpy as np
import os
import pandas as pd
import platform
from aicsimageio.aics_image import AICSImage
#from aicsimageio import AICSImage
from magicgui import magicgui
from pathlib import Path
from qtpy.QtWidgets import QVBoxLayout, QWidget, QLabel, QComboBox
from skimage.io import imread


In [2]:
PATH = 'Path'
DIR = 'Directory'
DATE = 'Date'
TIMEPOINT = 'TimePoint'
ZSTEP = 'ZStep'
PLATE = 'Plate'
WELL = 'Well'
SITE = 'Site'
CHANNEL = 'Channel'
UUID = 'UUID'

metadata_columns = {
    'mc2': TIMEPOINT,
    'mc3': ZSTEP,
    'mc4': PLATE,
    'mc5': WELL,
    'mc6': SITE,
    'mc7': CHANNEL,
}

def create_file_list(orig, ftype='tif'):
    print(orig)
    if not orig:
        return pd.DataFrame()
    
    files = [(str(x)) for x in orig.glob(f"*/*.{ftype}") if not "_flows" in x.name]
    df = pd.DataFrame(files, columns=[PATH])

    if not df.empty:
        print(files[-1])

        
    # Cross-platform pattern with dynamic column names
    pattern = (\
        r'[/\\](?P<{mc4}>[^/\\]*)'\
        r'[/\\]Well_(?P<{mc5}>[A-Z]\d*)_(?P<{mc6}>\d*)'\
        + r'[/\\](?P<{mc2}>\d\d\d\d_\d\d_\d\d_\d\d_\d\d_\d\d_\d\d).*[tif,jpg,png]'\
    ).format(**metadata_columns)

    #print(pattern)
    
    # Apply the regex pattern and extract the desired columns
    df_extracted = df[PATH].str.extract(pattern)
    #print()

    # Add the extracted columns back to the original dataframe
    df = df.join(df_extracted)
    
    #print(df.head(1))
    df[DIR] = df[PATH].apply(lambda x: str(Path(x).parent))
    df[PLATE] = df[PLATE].astype(str)
    df[WELL] = df[WELL].astype(str)
    df[SITE] = df[SITE].astype(int)
    #df[CHANNEL] = 1
    #df[ZSTEP] = 1
    
    return df


In [5]:
def create_dask_array(grouped2d):
    # Dictionary to store Dask arrays for each plate
    plates = []
    plate_stack = None

    # Prebuild index mapping
    index_map = {}
    index_map_reverse = {}

    # Group by plate and well to handle multiple sites within a well
    for plate, plate_group in grouped2d.groupby(PLATE):
        wells = []

        # Iterate over each well
        for well, well_group in plate_group.groupby(WELL):
            sites = []

            # Iterate over each site
            for site, site_group in well_group.groupby(SITE):
                # At this point, we know the plate, well, and site
                # Add an entry to index_mapping for this site
                index_map[(plate, well, site)] = (len(plates), len(wells), len(sites))
                index_map_reverse[(len(plates), len(wells), len(sites))] = (plate, well, site)

                #print('site_group')
                #print(site_group.columns)
                #print(site_group.shape)

                # Explode list columns
                exploded_site_group = site_group.explode([PATH])
                #print('exploded_site_group')
                #print(exploded_site_group.shape)
                #print(exploded_site_group.apply(type).unique())
                #print(exploded_site_group.head())

                channels = []

                # Iterate over each channel and stack them for the current Z-step
                for channel_path in exploded_site_group[PATH]:
                    print(plate, well, site, channel_path)
                    img = AICSImage(channel_path)
                    # Use img.get_image_dask_data() for lazy loading of data
                    dask_data = img.get_image_dask_data()
                    #print(dask_data.shape)
                    dask_data = dask_data.squeeze()
                    #print(dask_data.shape)
                    channels.append(dask_data)
                #print()
                
                # Stack channels for the current site
                site_stack = da.stack(channels, axis=0)  # Stack Z-slices to form a 3D site-level array
                #print(site_stack.shape)
                sites.append(site_stack)

            # Stack all site-level arrays into a well-level array
            well_stack = da.stack(sites, axis=0)  # Stack sites into a well
            wells.append(well_stack)

        # Stack all well-level arrays into a plate-level array
        plate_stack = da.stack(wells, axis=0)  # Stack wells into a plate
        plates.append(plate_stack)

    final_dask_array = da.stack(plates)
    return index_map, index_map_reverse, final_dask_array

def save_empty_label_image(folder, timestamp, extension, shape=(1040, 1392)):
    # Create an empty (black) image
    image = np.zeros(shape, dtype=np.uint8)  # uint8 ensures valid pixel values (0-255)

    path = str(Path(folder) / (timestamp + extension))
    print(path)
    
    # Convert to PIL Image and save
    imageio.imwrite(path, image)


# First create viewer, so it can be used in NavigationWidget
viewer = napari.Viewer()

# Create a widget for navigation
IDX_WELL = 1
class NavigationWidget(QWidget):
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        
        # Well selection
        self.well_label = QLabel("Well")
        self.well_combo = QComboBox()
        self.well_combo.currentTextChanged.connect(self.update_image)

        # Adding widgets to layout
        layout.addWidget(self.well_label)
        layout.addWidget(self.well_combo)

        self.setLayout(layout)
        
        viewer.dims.events.point.connect(self._update_display)
        
    def update_wells(self, wells):
        self.wells = wells
        self.well_combo.clear()
        self.well_combo.addItems(wells)
        
    def _update_display(self):
        #print("_update_display")
        slider_index = viewer.dims.point[IDX_WELL]
        slider_index = round(slider_index)
        self.well_combo.setCurrentText(self.wells[slider_index])

    def update_image(self):
        well = self.well_combo.currentText()
        #print(well)  # Debugging print

        # Select data based on plate, well, and site
        if well in self.wells:
            viewer.dims.set_point(IDX_WELL, self.wells.index(well))

navigate_wells = NavigationWidget()

def get_lmu_active1():
    current_os = platform.system()
    
    if current_os == "Windows":
        return "L:\\lmu_active1"
    elif current_os == "Linux":
        return "/mnt/lmu_active1"
    else:
        raise ValueError(f"Unsupported operating system: {current_os}")
    
# Store the last selected folder
last_selected_folder = Path(get_lmu_active1()) / 'instruments/Micro'  # Default to home directory
last_selected_folder = Path('/home/hajaalin/data/vizcaino/test')
last_selected_folder_labels = Path(get_lmu_active1()) / 'instruments/Micro'  # Default to home directory
last_selected_folder_labels = Path('/home/hajaalin/data/vizcaino/test_output_gpt')

# Store dataframe with images
df_images = pd.DataFrame()
df_images_grouped = pd.DataFrame()
df_labels = pd.DataFrame()
df_missing_labels = pd.DataFrame()
index_map_reverse = {}


def map_index_to_filename(index):
    #print(f'map_index_to_filename {index}')
    try:
        # Extract Plate, Well, and Site from the Napari index
        plate_idx, well_idx, site_idx, path_idx, _, _ = index  # Ignore last two indices

        plate, well, site = index_map_reverse[(plate_idx, well_idx, site_idx)]
        #print(f'plate, well, site: {plate}, {well}, {site}')

        # Find the row using MultiIndex lookup
        row = df_images_grouped.loc[(plate, well, site)]

        # Get the Path list from the dataframe
        path_list = row[PATH]
        #print(f'{path_list}')

        # Select the correct Timepoint using time_idx
        selected_path = path_list[path_idx]
        #print(f'{selected_path}')

        return selected_path

    except (KeyError, IndexError):
        return "Unknown file"


def update_filename(event):
    current_index = tuple(viewer.dims.current_step)  # Get current slider positions
    #print(f"Current index: {current_index}")

    # Map index to filename using your dataframe
    filename = map_index_to_filename(current_index)  # Define this function

    # Display filename
    viewer.text_overlay.visible = True
    viewer.text_overlay.text = filename  # Simple overlay in viewer


@magicgui(
    folder={"label": "Select Folder (images)", "mode": "d", "value": last_selected_folder},  # "d" stands for directory
)
def select_folder(folder: Path):
    if not folder or not folder.exists():
        print("Invalid folder")
        return
    
    last_selected_folder = folder  # Store the selected folder
    
    # create data frame with file list
    df = create_file_list(folder)
    df.sort_values(by=[PLATE, WELL, SITE, TIMEPOINT], inplace=True, ignore_index=True)
    
    global df_images
    global df_images_grouped
    df_images = df.copy()
    global index_map_reverse
    
    viewer.layers.clear()  # Clear existing layers

    if not df.empty:
        plates = list(df[PLATE].unique())
        wells = list(df[WELL].unique())
        sites = list(df[SITE].unique())
        
        # Update the well dropdown
        navigate_wells.update_wells(wells)

        grouped_df = df.groupby(by=[PLATE, WELL, SITE]).agg(list)
        index_map, index_map_reverse, final_dask_array_2d = create_dask_array(grouped_df)
        
        df_images_grouped = grouped_df.copy()

        viewer.add_image(
            final_dask_array_2d, 
            name = 'original',
        )

        viewer.dims.axis_labels = ['Plate', 'Site', 'Timepoint', 'X', 'Y']

        # start from Z-slice 0 to have labels visible
        # start from well 0 to match pull-down
        # start from site 0
        for i in range(len(viewer.dims.point)):
            print(i)
            viewer.dims.set_point(i,0)
            
        # Connect to Napari slider updates
        viewer.dims.events.current_step.connect(update_filename)

    else:
        print('No images found (stacks.empty)')



@magicgui(
    folder={"label": "Select Folder (labels)", "mode": "d", "value": last_selected_folder_labels},  # "d" stands for directory
)
def select_folder_labels(folder: Path):
    if not folder or not folder.exists():
        print("Invalid folder")
        return
    
    last_selected_folder_labels = folder  # Store the selected folder
    
    # create data frame with file list
    df = create_file_list(folder, ftype='png')
    df.sort_values(by=[PLATE, WELL, SITE, TIMEPOINT], inplace=True, ignore_index=True)
    
    global df_labels
    global df_missing_labels
    df_labels = df.copy()
    
    # find images without labels
    diff = df_images.merge(df, on=[WELL, SITE, TIMEPOINT], how='left', indicator=True, suffixes=('', '_labels'))
    df_missing_labels = diff[diff['_merge'] == 'left_only'].drop(columns=['_merge'])
        
    if not df_missing_labels.empty:
        print('missing labels')
        print(df_missing_labels.columns)
        print(df_missing_labels[[WELL, SITE, TIMEPOINT]])
        
        print('adding empty label images')
        for index, row in df_missing_labels.iterrows():
            folder = None
            mask = (df[WELL] == row[WELL]) & (df[SITE] == row[SITE])
            folder = df[mask][DIR].unique()[0]
            if folder:
                save_empty_label_image(folder, row[TIMEPOINT], '_dummy_labels.png')
                new_row = {PLATE: row[PLATE], WELL: row[WELL], SITE: row[SITE], TIMEPOINT: row[TIMEPOINT],\
                           PATH: folder}
                df_labels.loc[len(df_labels)] = new_row
                df_labels.sort_values(by=[PLATE, WELL, SITE, TIMEPOINT], inplace=True, ignore_index=True)
            else:
                print('No labels found for ' + folder)
                return
        return
        
    
    # remove previous label layer
    if len(viewer.layers) > 1:
        viewer.layers.pop()

    if not df.empty:
        plates = list(df[PLATE].unique())
        wells = list(df[WELL].unique())
        sites = list(df[SITE].unique())
        
        # Update the well dropdown
        #navigate_wells.update_wells(wells)

        grouped_df = df.groupby(by=[PLATE, WELL, SITE]).agg(list)
        index_map, index_map_reverse, final_dask_array_2d = create_dask_array(grouped_df)
        
        # add empty images for missing labels

        viewer.add_labels(
            final_dask_array_2d, 
            name = 'cellpose',
        )

        viewer.dims.axis_labels = ['Plate', 'Site', 'Timepoint', 'X', 'Y']

        # start from Z-slice 0 to have labels visible
        # start from well 0 to match pull-down
        # start from site 0
        for i in range(len(viewer.dims.point)):
            print(i)
            viewer.dims.set_point(i,0)
            
    else:
        print('No label images found (stacks.empty)')


viewer.window.add_dock_widget(select_folder)
viewer.window.add_dock_widget(select_folder_labels)
viewer.window.add_dock_widget(navigate_wells)

napari.run()

/home/hajaalin/data/vizcaino/test
/home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_29_21_98.tif
[/\\](?P<Plate>[^/\\]*)[/\\]Well_(?P<Well>[A-Z]\d*)_(?P<Site>\d*)[/\\](?P<TimePoint>\d\d\d\d_\d\d_\d\d_\d\d_\d\d_\d\d_\d\d).*[tif,jpg,png]

                                                Path Plate Well Site  \
0  /home/hajaalin/data/vizcaino/test/Well_A2_02/2...  test   A2   02   

                TimePoint  
0  2025_02_11_15_29_31_65  
test A2 1 /home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_29_21_98.tif
test A2 1 /home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_34_23_07.tif
test A2 1 /home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_39_21_12.tif
test A2 1 /home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_44_20_49.tif
test A2 1 /home/hajaalin/data/vizcaino/test/Well_A2_01/2025_02_11_15_49_19_65.tif
test A2 2 /home/hajaalin/data/vizcaino/test/Well_A2_02/2025_02_11_15_29_31_65.tif
test A2 2 /home/hajaalin/data/vizcaino/test/Well_A2_02/2025_0